# The Trail Foundation - Forecasting Trail Traffic

The Austin Parks department has time series data on foot and bike traffic on a trial near downtown Austin. This data is influenced by a number of conditions including weather, time of year, day of week, major and minor events, openings of new business and event spaces, and other more difficult-to-predict events and trends.

Using historical data, participants will attempt to develop models which capture these sources of variability in the data and forecast future expected traffic on various locations within the trails. This should include models, statistics, visualizations, and, potentially, interfacing components to allow models to be retrained and/or queried.

![image.png](attachment:image.png)

<img src="https://5107083.toastmastersclubs.org/imageuploads/5107083/walmart_technology_logo.png"/>

# 0. Imports

This is where we import the libraries we need for this exercise.

In [1]:
import numpy as np
import pandas as pd
from plotly import tools
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, iplot_mpl
import plotly.plotly as py
from scipy.optimize import leastsq

init_notebook_mode(connected=True)

# 1. Data Ingestion

This section contains all the necessary data ingestion code which allows for easier Exploratory Data Analysis later in the file.

## 1.1 Data Importing

The data is contained within the *daily_counts_7-2-19.xlsx* file. We import this and observe that the data was parsed correctly.

In [2]:
trail_df = pd.read_csv('daily_counts_10-25-19.csv')

trail_df = trail_df.rename(index=str, columns={"Time": "Date"})
trail_df.columns = ['Date', 'Butler Trail - Crenshaw Bridge PC Urban Trail', 
                    'Butler Trail - South Lamar PC Urban Trail', 'Butler Trail - North Congress PC Urban Trail',
                   'Butler Trail - Longhorn Dam PC Urban Trail', 'Shoal Creek Solar Trail PC Urban Trail ped/bike']

trail_df

,Date,Butler Trail - Crenshaw Bridge PC Urban Trail,Butler Trail - South Lamar PC Urban Trail,Butler Trail - North Congress PC Urban Trail,Butler Trail - Longhorn Dam PC Urban Trail,Shoal Creek Solar Trail PC Urban Trail ped/bike
0,2/17/16 0:00,4242.0,NaN,NaN,NaN,NaN
1,2/18/16 0:00,4979.0,NaN,NaN,NaN,NaN
2,2/19/16 0:00,5002.0,NaN,NaN,NaN,NaN
3,2/20/16 0:00,7697.0,NaN,NaN,NaN,NaN
4,2/21/16 0:00,5958.0,NaN,NaN,NaN,NaN
5,2/22/16 0:00,4289.0,NaN,NaN,NaN,NaN
6,2/23/16 0:00,2337.0,NaN,NaN,NaN,NaN
7,2/24/16 0:00,3888.0,NaN,NaN,NaN,NaN
8,2/25/16 0:00,4563.0,NaN,NaN,NaN,NaN
9,2/26/16 0:00,4266.0,NaN,NaN,NaN,NaN


## 1.2 Data Cleaning

There are several validations and cleaning steps we should do before analyzing the data. Let's take a look at some descriptive statistics to make sure the ranges for the column values are reasonable and see how the NaNs are being handled.

In [3]:
trail_df.shape

(1346, 6)

In [4]:
trail_df.describe()

,Butler Trail - Crenshaw Bridge PC Urban Trail,Butler Trail - South Lamar PC Urban Trail,Butler Trail - North Congress PC Urban Trail,Butler Trail - Longhorn Dam PC Urban Trail,Shoal Creek Solar Trail PC Urban Trail ped/bike
count,1297.000000,571.000000,567.000000,539.000000,807.000000
mean,4021.160370,2554.957968,2985.592593,948.474954,266.263941
std,2480.562293,1613.786218,1333.612252,470.920084,266.157994
min,224.000000,94.000000,332.000000,1.000000,1.000000
25%,2721.000000,1648.500000,2044.500000,730.500000,124.500000
50%,3801.000000,2278.000000,2792.000000,910.000000,191.000000
75%,4906.000000,3277.000000,3655.000000,1202.000000,329.000000
max,20530.000000,21729.000000,10374.000000,2569.000000,3061.000000


This looks right. `trail_df` is 1346 rows long, and the most entries any column has is 1297. Some have far fewer, likely because there wasn't a counter installed on the trail yet. It also appears that Pandas ignores NaNs instead of including them in counts or infilling them. I'm adding a month column since we will want to use that later.

In [5]:
trail_df['Date'] = pd.to_datetime(trail_df['Date'])

In [6]:
trail_df['Butler Trail - Crenshaw Bridge PC Urban Trail'] = trail_df['Butler Trail - Crenshaw Bridge PC Urban Trail'].replace(0, np.nan)
trail_df['month'] = trail_df['Date'].dt.month
trail_df['Butler Trail - Crenshaw Bridge PC Urban Trail'] = trail_df['Butler Trail - Crenshaw Bridge PC Urban Trail'].astype(float)

# trail_df = trail_df.astype({'Butler Trail - Crenshaw Bridge PC Urban Trail': 'int64', 'Butler Trail - South Lamar PC Urban Trail': 'Int64',
#                 'Butler Trail - North Congress PC Urban Trail': 'Int64', 'Butler Trail - Longhorn Dam PC Urban Trail': 'Int64',
#                 'Shoal Creek Solar Trail PC Urban Trail ped/bike': 'Int64', 'month': 'Int64'})

In [7]:
trail_df.dtypes

Date                                               datetime64[ns]
Butler Trail - Crenshaw Bridge PC Urban Trail             float64
Butler Trail - South Lamar PC Urban Trail                 float64
Butler Trail - North Congress PC Urban Trail              float64
Butler Trail - Longhorn Dam PC Urban Trail                float64
Shoal Creek Solar Trail PC Urban Trail ped/bike           float64
month                                                       int64
dtype: object

# 2. Exploratory Data Analysis (EDA)

This section contains basic EDA of the counts in the data set in order to understand their properties.

First, let's take a look about daily traffic at the five locations where there are people counters.

In [8]:
x = trail_df['Date']
y1 = trail_df['Butler Trail - Crenshaw Bridge PC Urban Trail']
y2 = trail_df['Butler Trail - South Lamar PC Urban Trail']
y3 = trail_df['Butler Trail - North Congress PC Urban Trail']
y4 = trail_df['Butler Trail - Longhorn Dam PC Urban Trail']
y5 = trail_df['Shoal Creek Solar Trail PC Urban Trail ped/bike']

trace1 = go.Scatter(x=x, y=y1, name='Butler - Crenshaw')
trace2 = go.Scatter(x=x, y=y2, name='Butler - S Lamar')
trace3 = go.Scatter(x=x, y=y3, name='Butler - Congress')
trace4 = go.Scatter(x=x, y=y4, name='Butler - Longhorn Dam')
trace5 = go.Scatter(x=x, y=y5, name='Shoal Creek')

data = [trace1, trace2, trace3, trace4, trace5]
layout = go.Layout(title='Daily Total Traffic', legend=dict(x=-.1, y=1.2))

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [9]:
# There are still some dates that are empty for one or more trails. Replacing this with a zero or an average would 
# indicate a trend that isn't there, so I will leave them as is.
trail_df[trail_df['Date'] == '2018-07-25']

,Date,Butler Trail - Crenshaw Bridge PC Urban Trail,Butler Trail - South Lamar PC Urban Trail,Butler Trail - North Congress PC Urban Trail,Butler Trail - Longhorn Dam PC Urban Trail,Shoal Creek Solar Trail PC Urban Trail ped/bike,month
889,2018-07-25,NaN,1732.0,1782.0,808.0,NaN,7


It will also be useful to get an idea how traffic on the trail varies by day. Let's look at mean traffic counts for each of those five locations by day of the week.

The output row indices indicate day of the week from Mon - Sun.

In [10]:
week_df = trail_df.groupby(trail_df['Date'].dt.weekday).mean().drop(columns=['month'])
week_df

,Butler Trail - Crenshaw Bridge PC Urban Trail,Butler Trail - South Lamar PC Urban Trail,Butler Trail - North Congress PC Urban Trail,Butler Trail - Longhorn Dam PC Urban Trail,Shoal Creek Solar Trail PC Urban Trail ped/bike
Date,,,,,
0,3415.827027,2425.469136,2639.358025,831.506329,244.922414
1,3316.075676,2044.086420,2472.938272,765.558442,247.478261
2,3364.897849,1929.439024,2379.864198,741.743590,258.052174
3,3355.284946,2129.048780,2648.950617,789.924051,289.808696
4,3602.589189,1997.012195,2667.160494,767.828947,267.905172
5,5675.248649,3631.585366,4122.000000,1376.720000,291.521739
6,5425.345946,3734.641975,3968.876543,1396.293333,264.330435


In [11]:
x = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

trace1 = go.Bar(x=x, y=week_df['Butler Trail - Crenshaw Bridge PC Urban Trail'], name='Butler - Crenshaw')
trace2 = go.Bar(x=x, y=week_df['Butler Trail - South Lamar PC Urban Trail'], name='Butler - South Lamar')
trace3 = go.Bar(x=x, y=week_df['Butler Trail - North Congress PC Urban Trail'], name='Butler - Congress')
trace4 = go.Bar(x=x, y=week_df['Butler Trail - Longhorn Dam PC Urban Trail'], name='Butler - Longhorn Dam')
trace5 = go.Bar(x=x, y=week_df['Shoal Creek Solar Trail PC Urban Trail ped/bike'], name='Shoal Creek')

data = [trace1, trace2, trace3, trace4, trace5]
layout = go.Layout(xaxis=dict(tickangle=-45), barmode='group', title='Mean Traffic by Weekday')

fig = go.Figure(data=data, layout=layout)
iplot(fig)

Weekends are busier on average for every section of the trail that TTF monitors. This is what we would expect. 

Now let's look to see if there is monthly periodicity too. The output row indices indicate month from Jan - Dec.

In [12]:
month_df = trail_df.groupby(trail_df['Date'].dt.month).mean().drop(columns=['month'])

month_df

,Butler Trail - Crenshaw Bridge PC Urban Trail,Butler Trail - South Lamar PC Urban Trail,Butler Trail - North Congress PC Urban Trail,Butler Trail - Longhorn Dam PC Urban Trail,Shoal Creek Solar Trail PC Urban Trail ped/bike
Date,,,,,
1,4143.634409,2772.903226,2842.516129,946.483871,126.138889
2,4164.886598,2764.285714,3044.642857,725.562500,125.479167
3,4681.104839,2978.031250,4004.864865,755.545455,128.483871
4,4445.216667,3542.083333,4196.383333,1132.728814,397.579545
5,3903.661290,3109.983871,2820.354839,1082.016129,309.720430
6,3336.948980,2755.716667,3221.400000,1121.983333,341.500000
7,2801.010309,2427.661290,2783.048387,1080.887097,302.369863
8,2800.443548,1824.612903,2199.870968,987.032258,238.725000
9,3330.583333,1991.350000,2641.333333,1005.850000,292.753247


In [13]:
x = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November',
    'December']

trace1 = go.Bar(x=x, y=month_df['Butler Trail - Crenshaw Bridge PC Urban Trail'], name='Butler - Crenshaw')
trace2 = go.Bar(x=x, y=month_df['Butler Trail - South Lamar PC Urban Trail'], name='Butler - South Lamar')
trace3 = go.Bar(x=x, y=month_df['Butler Trail - North Congress PC Urban Trail'], name='Butler - Congress')
trace4 = go.Bar(x=x, y=month_df['Butler Trail - Longhorn Dam PC Urban Trail'], name='Butler - Longhorn Dam')
trace5 = go.Bar(x=x, y=month_df['Shoal Creek Solar Trail PC Urban Trail ped/bike'], name='Shoal Creek')

data = [trace1, trace2, trace3, trace4, trace5]
layout = go.Layout(xaxis=dict(tickangle=-45), barmode='group', title='Mean Traffic by Month')

fig = go.Figure(data=data, layout=layout)
iplot(fig)

Highest traffic for most locations is in March/April, possibly because that's when the nice spring weather starts. Peak traffic at Butler-Crenshaw is in October - this is likely caused by ACL, which is in October and is [right next door](https://goo.gl/maps/Qf9k7HQpVvMApN8z8) to the counter.

Let's see if there is periodicity between weeks of the year. The output row indices indicate month from Jan - Dec.

In [14]:
week_df = trail_df.groupby(trail_df['Date'].dt.week).mean().drop(columns=['month'])
week_df = week_df[:-1]
week_df

,Butler Trail - Crenshaw Bridge PC Urban Trail,Butler Trail - South Lamar PC Urban Trail,Butler Trail - North Congress PC Urban Trail,Butler Trail - Longhorn Dam PC Urban Trail,Shoal Creek Solar Trail PC Urban Trail ped/bike
Date,,,,,
1,3964.047619,3263.142857,3809.000000,1096.142857,82.000000
2,4336.000000,3213.857143,2862.000000,1016.571429,143.142857
3,3593.809524,2371.000000,2391.714286,803.428571,101.714286
4,4720.761905,2843.285714,2973.714286,1102.857143,175.200000
5,4291.190476,2782.000000,2540.142857,961.714286,88.857143
6,3713.380952,2037.285714,2719.571429,721.714286,118.785714
7,4327.846154,3075.857143,3551.000000,457.600000,193.000000
8,4351.607143,3115.714286,3241.857143,5.000000,90.888889
9,4874.678571,2136.428571,2241.000000,NaN,104.818182


In [15]:
x = [i for i in range(52)]

trace1 = go.Bar(x=x, y=week_df['Butler Trail - Crenshaw Bridge PC Urban Trail'], name='Butler - Crenshaw')
trace2 = go.Bar(x=x, y=week_df['Butler Trail - South Lamar PC Urban Trail'], name='Butler - South Lamar')
trace3 = go.Bar(x=x, y=week_df['Butler Trail - North Congress PC Urban Trail'], name='Butler - Congress')
trace4 = go.Bar(x=x, y=week_df['Butler Trail - Longhorn Dam PC Urban Trail'], name='Butler - Longhorn Dam')
trace5 = go.Bar(x=x, y=week_df['Shoal Creek Solar Trail PC Urban Trail ped/bike'], name='Shoal Creek')

data = [trace1, trace2, trace3, trace4, trace5]

layout = go.Layout(xaxis=dict(tickvals=[(2*k-1)*(52/12)/2 for k in range(1,13)],
                              ticktext=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
                                        'September', 'October', 'November', 'December'], 
                              tickangle=-45), 
                   yaxis=dict(hoverformat='.0f'), 
                   barmode='group', 
                   title='Mean Traffic by Ordinal Week',
                   annotations=[dict(x=39, y=9400, 
                                     xref='x', yref='y', 
                                     text='ACL Weekends', 
                                     showarrow=True, arrowhead=6, 
                                     ax=0, ay=-30)]
                  )


fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [16]:
# Box plot of Butler-Crenshaw traffic by month.

x = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November',
    'December']

trace_dict = {}
for i in range(len(x)):
    trace_dict['trace{}'.format(i)] = go.Box(
        {
        'y': trail_df['Butler Trail - Crenshaw Bridge PC Urban Trail'].loc[trail_df['month'] == i + 1], 
        'type':'box', 
        'name': x[i]
        }
    )

data = [data for trace, data in trace_dict.items()]
layout = go.Layout(xaxis=dict(tickangle=-45), title='Monthly Count Box Plots (Butler-Crenshaw)', showlegend=False)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

# 3. Forecasting

I'm now going to test out the Prophet library from Facebook. This will let me model the trends we are seeing, add information about known busy times on the trail, and add weather as a regressor. I'll just be looking at the Crenshaw Bridge counts since they cover the longest time period and have the most traffic. I'll be trying out several models and evaluating their performance against each other:
* Mean
* Moving Average
* ARIMA
* Generalized Additive Model (GAM)

## 3.1 Data Processing
These four models need the data in different formats. I'll go ahead and take care of that here so we don't have to do it later.

In [17]:
from data_imputation import impute_data

from matplotlib import pyplot
from matplotlib import figure

from pandas import read_csv
from pandas import datetime
from pandas.plotting import autocorrelation_plot

from statsmodels.tsa.arima_model import ARIMA

from sklearn.metrics import mean_squared_error, mean_absolute_error


%load_ext autoreload

%autoreload 2

I want to only work with the period that has values for Butler-Crenshaw since that goes back the farthest. Before I start building and evaluating models on that data, I need to decide how to take care of the missing values from 6/9/18 - 7/27/18. I don't want to lose 3 years of data by starting at the end of this, so I'm going to put in average values for the missing dates (accounting for the day of the week that it is).

In [18]:
crenshaw_df = trail_df.copy()
crenshaw_df['week'] = crenshaw_df['Date'].dt.week
crenshaw_df['day'] = crenshaw_df['Date'].dt.dayofweek

In [19]:
crenshaw_df.head()

,Date,Butler Trail - Crenshaw Bridge PC Urban Trail,Butler Trail - South Lamar PC Urban Trail,Butler Trail - North Congress PC Urban Trail,Butler Trail - Longhorn Dam PC Urban Trail,Shoal Creek Solar Trail PC Urban Trail ped/bike,month,week,day
0,2016-02-17,4242.0,NaN,NaN,NaN,NaN,2,7,2
1,2016-02-18,4979.0,NaN,NaN,NaN,NaN,2,7,3
2,2016-02-19,5002.0,NaN,NaN,NaN,NaN,2,7,4
3,2016-02-20,7697.0,NaN,NaN,NaN,NaN,2,7,5
4,2016-02-21,5958.0,NaN,NaN,NaN,NaN,2,7,6


In [20]:
nan_idxs = list(np.where(crenshaw_df['Butler Trail - Crenshaw Bridge PC Urban Trail'].isnull())[0])

For every row in nan_idxs:
* Select all *other* (excluding current) instances in the dataset with that same week and day.
* Take the mean count for those days for Crenshaw
* Replace count (currently NaN) for row with mean count

In [21]:
inverse_df = crenshaw_df.drop(crenshaw_df.index[nan_idxs])

In [22]:
def impute_nans(nan_row):
    week = nan_row['week']
    day = nan_row['day']
    temp_df = inverse_df.query('week=={} and day=={}'.format(week, day))
    new_val = temp_df['Butler Trail - Crenshaw Bridge PC Urban Trail'].mean()
    return new_val

In [23]:
nan_df = crenshaw_df.iloc[nan_idxs]
nan_df['Butler Trail - Crenshaw Bridge PC Urban Trail'] = nan_df.apply(impute_nans, axis=1)

/anaconda3/envs/ttf_hackathon/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [24]:
# NaN values are now replaced with the imputed values created above.
crenshaw_df.iloc[nan_idxs] = nan_df

In [25]:
# Yo.
crenshaw_df.iloc[nan_idxs]

,Date,Butler Trail - Crenshaw Bridge PC Urban Trail,Butler Trail - South Lamar PC Urban Trail,Butler Trail - North Congress PC Urban Trail,Butler Trail - Longhorn Dam PC Urban Trail,Shoal Creek Solar Trail PC Urban Trail ped/bike,month,week,day
843,2018-06-09,4883.000000,4231.0,4363.0,1559.0,495.0,6,23,5
844,2018-06-10,4303.333333,3310.0,3720.0,1629.0,423.0,6,23,6
845,2018-06-11,2832.666667,1931.0,2510.0,1208.0,371.0,6,24,0
846,2018-06-12,3314.000000,2001.0,2615.0,986.0,363.0,6,24,1
847,2018-06-13,4594.666667,2232.0,2616.0,984.0,343.0,6,24,2
848,2018-06-14,2913.666667,2214.0,2846.0,1761.0,510.0,6,24,3
849,2018-06-15,2649.666667,2223.0,2945.0,1026.0,302.0,6,24,4
850,2018-06-16,3903.000000,4130.0,4457.0,1453.0,406.0,6,24,5
851,2018-06-17,3488.666667,3965.0,3957.0,1466.0,484.0,6,24,6
852,2018-06-18,2901.333333,2260.0,2696.0,717.0,489.0,6,25,0


In [26]:
# Let's take a look and make sure this looks sane.
x = crenshaw_df['Date']
y = crenshaw_df['Butler Trail - Crenshaw Bridge PC Urban Trail']

trace1 = go.Scatter(x=x, y=y, name='Butler - Crenshaw')

data = [trace1]
layout = go.Layout(title='Daily Total Traffic', legend=dict(x=-.1, y=1.2))

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [27]:
multivariate_df = crenshaw_df.copy()
multivariate_df = multivariate_df.set_index('Date')

weather_df = pd.read_csv('weather_10-22-19.csv')
weather_df['DATE'] = pd.to_datetime(weather_df['DATE'])
weather_df = weather_df.set_index('DATE')

multivariate_df = pd.merge(multivariate_df, weather_df, how='inner', left_index=True, right_index=True)

multivariate_df = multivariate_df.drop(columns=['month', 'week', 'day', 'STATION', 'NAME', 
                                                'Butler Trail - South Lamar PC Urban Trail', 
                                                'Butler Trail - North Congress PC Urban Trail', 
                                                'Butler Trail - Longhorn Dam PC Urban Trail', 
                                                'Shoal Creek Solar Trail PC Urban Trail ped/bike'])

multivariate_df.head()

,Butler Trail - Crenshaw Bridge PC Urban Trail,PRCP,TMAX,TMIN
2016-02-17,4242.0,0.0,79,45
2016-02-18,4979.0,0.0,73,46
2016-02-19,5002.0,0.0,77,53
2016-02-20,7697.0,0.0,79,62
2016-02-21,5958.0,0.0,73,63


In [28]:
multivariate_df.tail()

,Butler Trail - Crenshaw Bridge PC Urban Trail,PRCP,TMAX,TMIN
2019-10-18,2549.0,0.0,74,47
2019-10-19,2707.0,0.0,82,48
2019-10-20,2510.0,0.0,92,61
2019-10-21,2839.0,0.4,90,59
2019-10-22,3101.0,0.0,76,48


Now we have a univariate timeseries with no missing values and outliers removed.

In [29]:
# split a dataset into train/test sets
def split_dataset(data):
    # split into standard weeks
    test_split_loc = .2 * len(data)
    test_split_loc = int(7 * (test_split_loc // 7)) - 1
    train, test = data[2:-test_split_loc], data[-test_split_loc:-6]    
    # restructure into windows of weekly data
    train = np.array(np.split(train, len(train)/7))
    test = np.array(np.split(test, len(test)/7))
    return train, test

In [30]:
train, test = split_dataset(multivariate_df['Butler Trail - Crenshaw Bridge PC Urban Trail'])

ValueError: array split does not result in an equal division

In [31]:
days = ['+1', '+2', '+3', '+4', '+5', '+6', '+7']

### 3.1.1 Mean Model

In [ ]:
split = int(len(multivariate_df) * .8)

train, test = multivariate_df[:split], multivariate_df[split:]

In [ ]:
mean = train['Butler Trail - Crenshaw Bridge PC Urban Trail'][-7:].mean()

In [ ]:
def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

def mean_scoring(mean, train_data, test_data, forecast_len=7):
    predictions_list = []
    predictions = [mean for i in range(forecast_len)]
    predictions_list.append(predictions)
    data = train_data.append(test_data)
    for i in range(len(test_data) - forecast_len):
        mean = data['Butler Trail - Crenshaw Bridge PC Urban Trail'][len(train_data) - 6 + i:len(train_data) - 6 + i + forecast_len].mean()
        predictions = [mean for i in range(forecast_len)]
        predictions_list.append(predictions)
    # calculate mae
    test_weeks = rolling_window(test['Butler Trail - Crenshaw Bridge PC Urban Trail'], 7)
    predictions_array = np.array(predictions_list)
    scores = []
    for i in range(test_weeks.shape[1]):
        mae = mean_absolute_error(test_weeks[:, i],
                                  predictions_array[:, i])
        scores.append(mae)
    # calculate overall MAE
    s = 0
    for row in range(test_weeks.shape[0]):
        for col in range(test_weeks.shape[1]):
            s += np.abs((test_weeks[row, col] - predictions_array[row, col]))
    score = (s / (test_weeks.shape[0] * test_weeks.shape[1]))
    return score, scores, predictions_list

In [ ]:
mean_score, mean_scores, mean_pred_list = mean_scoring(mean, train, test)

In [ ]:
print('Mean Model: [%.3f] %s' % (mean_score, mean_scores))


# forecast of final week of data compared to actual final week
test_trace = go.Scatter(
    x=test.index[-7:],
    y=test['Butler Trail - Crenshaw Bridge PC Urban Trail'][-7:],
    name='Actual Data'
)

forecast_trace = go.Scatter(
    x=test.index[-7:],
    y=mean_pred_list[-1],
    name='Forecasted Data'
)

data = [
    test_trace, 
    forecast_trace
]

layout = go.Layout(
    xaxis=dict(
        tickvals=test.index[-7:],
#         ticktext=days,
    ), 
    yaxis=dict(
        title='Count',
        rangemode='tozero'
    ), 
    title='Naive Mean Model Forecast'
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

# error plot
mae_trace = go.Scatter(
    x=days, 
    y=mean_scores, 
    name='Naive Mean'
)

data = [mae_trace]

layout = go.Layout(
    xaxis=dict(
        tickvals=days,
        ticktext=days,
        title='Days Forecast',
    ), 
    yaxis=dict(
        title='Mean Absolute Error',
        rangemode='tozero'
    ), 
    title='Naive Mean Model Performance'
)


fig = go.Figure(data=data, layout=layout)
iplot(fig)

### 3.1.2 Sine Wave Model

In [ ]:
t = np.array([i for i in range(len(train))])

guess_mean = np.mean(np.array(train['Butler Trail - Crenshaw Bridge PC Urban Trail']))
guess_std = 3*np.std(np.array(train['Butler Trail - Crenshaw Bridge PC Urban Trail']))/(2**0.5)/(2**0.5)
guess_phase = 0
guess_freq = 1
guess_amp = 1

# we'll use this to plot our first estimate. This might already be good enough for you
data_first_guess = guess_std*np.sin(t+guess_phase) + guess_mean

# Define the function to optimize, in this case, we want to minimize the difference
# between the actual data and our "guessed" parameters
optimize_func = lambda x: x[0]*np.sin(x[1]*t+x[2]) + x[3] - np.array(train['Butler Trail - Crenshaw Bridge PC Urban Trail'])
est_amp, est_freq, est_phase, est_mean = leastsq(optimize_func, [guess_amp, guess_freq, guess_phase, guess_mean])[0]

# recreate the fitted curve using the optimized parameters
data_fit = est_amp*np.sin(est_freq*t+est_phase) + est_mean

In [ ]:
def sine_scoring(train_data, test_data, forecast_len=7):
    predictions_list = []
    t = np.array([i for i in range(len(train_data) - forecast_len, len(train_data))])
    predictions = est_amp * np.sin(est_freq * t + est_phase) + est_mean
    predictions_list.append(predictions)
#     data = train_data.append(test_data)
    for i in range(len(test_data) - forecast_len):
        t += 1
        predictions = est_amp * np.sin(est_freq * t + est_phase) + est_mean
        predictions_list.append(predictions)
    # calculate mae
    test_weeks = rolling_window(test['Butler Trail - Crenshaw Bridge PC Urban Trail'], 7)
    predictions_array = np.array(predictions_list)
    scores = []
    for i in range(test_weeks.shape[1]):
        mae = mean_absolute_error(test_weeks[:, i],
                                  predictions_array[:, i])
        scores.append(mae)
    # calculate overall MAE
    s = 0
    for row in range(test_weeks.shape[0]):
        for col in range(test_weeks.shape[1]):
            s += np.abs((test_weeks[row, col] - predictions_array[row, col]))
    score = (s / (test_weeks.shape[0] * test_weeks.shape[1]))
    return score, scores, predictions_list

In [ ]:
sine_score, sine_scores, sine_pred_list = sine_scoring(train, test)

In [ ]:
print('Sine Wave Model: [%.3f] %s' % (sine_score, sine_scores))


t = np.array([i for i in range(7)])


# forecast of final week of data compared to actual final week
test_trace = go.Scatter(
    x=test.index[-7:],
    y=test['Butler Trail - Crenshaw Bridge PC Urban Trail'][-7:],
    name='Actual Data'
)

forecast_trace = go.Scatter(
    x=test.index[-7:], 
    y=sine_pred_list[-1],
    name='Forecasted Data'
)


data = [
    test_trace, 
    forecast_trace
]

layout = go.Layout(
    xaxis=dict(
        tickvals=test.index[-7:],
#         ticktext=days,
    ), 
    yaxis=dict(
        title='Count',
        rangemode='tozero'
    ), 
    title='Naive Sine Wave Model Forecast'
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

# error plot
mae_trace = go.Scatter(
    x=days, 
    y=sine_scores, 
    name='Naive Sine Wave'
)

data = [mae_trace]

layout = go.Layout(
    xaxis=dict(
        tickvals=days,
        ticktext=days,
        title='Days Forecast',
    ), 
    yaxis=dict(
        title='Mean Absolute Error',
        rangemode='tozero'
    ), 
    title='Naive Sine Wave Model Performance'
)


fig = go.Figure(data=data, layout=layout)
iplot(fig)

### 3.1.3 ARIMA Model

In [77]:
sunspot_data = pd.read_csv('monthly-sunspots.csv')
sunspot_data = np.array(sunspot_data['Sunspots'])
sunspot_data

array([58. , 62.6, 70. , ..., 55.8, 33.3, 33.4])

In [78]:
sunspot_data.shape

(2820,)

In [79]:
# split = int(len(multivariate_df) * .8)

# train, test = multivariate_df[:split], multivariate_df[split:]

split = int(len(sunspot_data) * .8)

train, test = sunspot_data[:split], sunspot_data[split:]

In [107]:
train

array([15136., 16733., 20016., 17708., 18019., 19227., 22893., 23739.,
       21133., 22591., 26786., 29740., 15028., 17977., 20008., 21354.,
       19498., 22125., 25817., 28779., 20960., 22254., 27392., 29945.,
       16933., 17892., 20533., 23569., 22417., 22084., 26580., 27454.,
       24081., 23451., 28991., 31386., 16896., 20045., 23471., 21747.,
       25621., 23859., 25500., 30998., 24475., 23145., 29701., 34365.,
       17556., 22077., 25702., 22214., 26886., 23191., 27831., 35406.,
       23195., 25110., 30009., 36242., 18450., 21845., 26488., 22394.,
       28057., 25451., 24872., 33424., 24052., 28449., 33533., 37351.,
       19969., 21701., 26249., 24493., 24603., 26485., 30723., 34569.,
       26689., 26157., 32064., 38870., 21337., 19419., 23166., 28286.,
       24570., 24001., 33151., 24878., 26804., 28967., 33311., 40226.,
       20504., 23060., 23562., 27562., 23940., 24584., 34303., 25517.,
       23494., 29095., 32903., 34379., 16991., 21109., 23740., 25552.,
      

In [115]:
arima_model = pm.auto_arima(
y=train,
seasonal=False,
trace=True,
scoring='mse',
suppress_warnings=True
)

print(arima_model)

sarima_model = pm.auto_arima(
y=train,
seasonal=True,
m=1,
trace=True,
scoring='mse',
suppress_warnings=True
)

print(sarima_model)

Fit ARIMA: order=(2, 1, 2); AIC=3266.897, BIC=3285.496, Fit time=0.156 seconds
Fit ARIMA: order=(0, 1, 0); AIC=3356.338, BIC=3362.538, Fit time=0.002 seconds
Fit ARIMA: order=(1, 1, 0); AIC=3344.232, BIC=3353.532, Fit time=0.036 seconds
Fit ARIMA: order=(0, 1, 1); AIC=3276.437, BIC=3285.737, Fit time=0.027 seconds
Fit ARIMA: order=(1, 1, 2); AIC=3270.990, BIC=3286.489, Fit time=0.110 seconds
Fit ARIMA: order=(3, 1, 2); AIC=3268.838, BIC=3290.537, Fit time=0.212 seconds
Fit ARIMA: order=(2, 1, 1); AIC=3264.951, BIC=3280.451, Fit time=0.100 seconds
Fit ARIMA: order=(1, 1, 1); AIC=3276.071, BIC=3288.470, Fit time=0.070 seconds
Fit ARIMA: order=(3, 1, 1); AIC=3266.858, BIC=3285.457, Fit time=0.173 seconds
Fit ARIMA: order=(2, 1, 0); AIC=3307.932, BIC=3320.331, Fit time=0.053 seconds
Total fit time: 0.941 seconds
ARIMA(callback=None, disp=0, maxiter=None, method=None, order=(2, 1, 1),
      out_of_sample_size=0, scoring='mse', scoring_args=None,
      seasonal_order=None, solver='lbfgs', st

In [85]:
import pmdarima as pm
from pmdarima import model_selection

arima_model = pm.auto_arima(
#                             y=np.array(train[['Butler Trail - Crenshaw Bridge PC Urban Trail']]).reshape(len(train),),
                            y=train,
#                             exogenous=np.array(train[['TMAX', 'TMIN', 'PRCP']]).reshape(len(train), 3),
                            seasonal=False,
#                             information_criterion='aic',
                            trace=True,
                            scoring='mae',
                            suppress_warnings=True, 
#                             stepwise=True
                           )

sarima_model = pm.auto_arima(
#                             y=np.array(train[['Butler Trail - Crenshaw Bridge PC Urban Trail']]).reshape(len(train),),
                            y=train,
#                              exogenous=np.array(train[['TMAX', 'TMIN', 'PRCP']]).reshape(len(train), 3),
                             m=12,
                             seasonal=True,
#                              information_criterion='aic',
                             scoring='mae',
                             trace=True, 
                             suppress_warnings=True, 
#                              stepwise=True
                            )

# print(arima_model.aic())

Fit ARIMA: order=(2, 1, 2); AIC=18610.541, BIC=18644.866, Fit time=0.597 seconds
Fit ARIMA: order=(0, 1, 0); AIC=19015.472, BIC=19026.914, Fit time=0.002 seconds
Fit ARIMA: order=(1, 1, 0); AIC=18779.195, BIC=18796.358, Fit time=0.023 seconds
Fit ARIMA: order=(0, 1, 1); AIC=18639.334, BIC=18656.496, Fit time=0.018 seconds
Fit ARIMA: order=(1, 1, 2); AIC=18608.014, BIC=18636.619, Fit time=0.175 seconds
Fit ARIMA: order=(1, 1, 1); AIC=18607.086, BIC=18629.970, Fit time=0.171 seconds
Fit ARIMA: order=(2, 1, 1); AIC=18607.443, BIC=18636.047, Fit time=0.227 seconds
Total fit time: 1.216 seconds
Fit ARIMA: order=(2, 1, 2) seasonal_order=(1, 0, 1, 12); AIC=18566.128, BIC=18611.895, Fit time=6.775 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 0, 0, 12); AIC=19015.472, BIC=19026.914, Fit time=0.044 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 0, 0, 12); AIC=18781.193, BIC=18804.077, Fit time=0.346 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 1, 12); AIC=18639.783, B

In [89]:
arima_model

ARIMA(callback=None, disp=0, maxiter=None, method=None, order=(1, 1, 1),
      out_of_sample_size=0, scoring='mae', scoring_args=None,
      seasonal_order=None, solver='lbfgs', start_params=None,
      suppress_warnings=True, transparams=True, trend=None,
      with_intercept=True)

In [68]:
model1 = pm.ARIMA(order=(5, 0, 5))
model2 = pm.ARIMA(order=(4, 0, 3), seasonal_order=(1, 0, 2, 7))

In [52]:
np.array(train[['Butler Trail - Crenshaw Bridge PC Urban Trail']]).reshape(len(train),).shape

(1075,)

In [116]:
cv = model_selection.SlidingWindowForecastCV(window_size=100, step=24, h=1)

arima_cv_scores = model_selection.cross_val_score(
    arima_model,
#     np.array(train[['Butler Trail - Crenshaw Bridge PC Urban Trail']]).reshape(len(train),),
    train,
    scoring='smape', 
    cv=cv, 
    verbose=2)

sarima_cv_scores = model_selection.cross_val_score(
    sarima_model, 
#     np.array(train[['Butler Trail - Crenshaw Bridge PC Urban Trail']]).reshape(len(train),),
    train,
    scoring='smape',
    cv=cv,
    verbose=2)

print("Model 1 CV scores: {}".format(arima_cv_scores.tolist()))
print("Model 2 CV scores: {}".format(sarima_cv_scores.tolist()))

# Pick based on which has a lower mean error rate
m1_average_error = np.nanmean(arima_cv_scores)
m2_average_error = np.nanmean(sarima_cv_scores)
errors = [m1_average_error, m2_average_error]
models = [arima_model, sarima_model]

# print out the answer
better_index = np.argmin(errors)  # type: int
print("Lowest average SMAPE: {} (model{})".format(
    errors[better_index], better_index + 1))
print("Best model: {}".format(models[better_index]))

[CV] fold=0 ..........................................................
[CV] fold=1 ..........................................................
[CV] fold=2 ..........................................................
[CV] fold=0 ..........................................................
[CV] fold=1 ..........................................................
[CV] fold=2 ..........................................................
Model 1 CV scores: [24.154273464044987, 21.44587848785966, 2.9016745203024974]
Model 2 CV scores: [23.928976841681518, 22.289666122815046, 3.7484080171047283]
Lowest average SMAPE: 16.167275490735715 (model1)
Best model: ARIMA(callback=None, disp=0, maxiter=None, method=None, order=(2, 1, 1),
      out_of_sample_size=0, scoring='mse', scoring_args=None,
      seasonal_order=None, solver='lbfgs', start_params=None,
      suppress_warnings=True, transparams=True, trend=None,
      with_intercept=True)


In [101]:
sarima_model.predict

ARIMA(callback=None, disp=0, maxiter=None, method=None, order=(3, 1, 2),
      out_of_sample_size=0, scoring='mae', scoring_args=None,
      seasonal_order=(0, 0, 0, 12), solver='lbfgs', start_params=None,
      suppress_warnings=True, transparams=True, trend=None,
      with_intercept=True)

In [ ]:
print(arima_model.summary())

In [99]:
def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

def arima_scoring(model, test_data, forecast_len=7):
    predictions_list = []
    predictions = model.predict(n_periods=forecast_len, 
#                                 exogenous=test_data[['TMAX', 'TMIN', 'PRCP']].iloc[0:0 + forecast_len]
                               )
    predictions_list.append(predictions)
    for i in range(len(test_data) - forecast_len):
        model = model.update(y=test_data[['Butler Trail - Crenshaw Bridge PC Urban Trail']].iloc[i],
#                              exogenous=np.array(test_data[['TMAX', 'TMIN', 'PRCP']].iloc[i + forecast_len]).reshape(1, -1)
                            )
        predictions = model.predict(n_periods=forecast_len, 
#                                     exogenous=test_data[['TMAX', 'TMIN', 'PRCP']].iloc[i + 1:i + forecast_len + 1]
                                   )
        predictions_list.append(predictions)
    # calculate mae
    test_weeks = rolling_window(
        test_data['Butler Trail - Crenshaw Bridge PC Urban Trail'], 
        7)
    predictions_array = np.array(predictions_list)
    scores = []
    for i in range(test_weeks.shape[1]):
        mae = mean_absolute_error(test_weeks[:, i],
                                  predictions_array[:, i])
        scores.append(mae)
    # calculate overall MAE
    s = 0
    for row in range(test_weeks.shape[0]):
        for col in range(test_weeks.shape[1]):
            s += np.abs((test_weeks[row, col] - predictions_array[row, col]))
    score = (s / (test_weeks.shape[0] * test_weeks.shape[1]))
    return score, scores, predictions_list

In [ ]:
arima_score, arima_scores, arima_pred_list = arima_scoring(arima_model, test)

In [ ]:
print('ARIMA Model: [%.3f] %s' % (arima_score, arima_scores))


arima_trace = go.Scatter(
    x=test.index[-7:], 
    y=arima_pred_list[-1], 
    name='Forecasted Data'
)

test_trace = go.Scatter(
    x=test.index[-7:], 
    y=test['Butler Trail - Crenshaw Bridge PC Urban Trail'][-7:], 
    name='Actual Data'
)

data = [arima_trace, test_trace]

layout = go.Layout(
    xaxis=dict(
        tickvals=test.index[-7:],
#         ticktext=days,
    ), 
    yaxis=dict(
        title='Count',
        rangemode='tozero'
    ), 
    title='ARIMA Model Forecast'
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

mae_trace = go.Scatter(
    x=days, 
    y=arima_scores,
    name='Mean Absolute Error'
)

data = [mae_trace]

layout = go.Layout(
    xaxis=dict(
        tickvals=days,
        ticktext=days,
        title='Days Forecast',
    ), 
    yaxis=dict(
        title='Mean Absolute Error',
        rangemode='tozero'
    ), 
    title='ARIMA Model Performance'
)


fig = go.Figure(data=data, layout=layout)
iplot(fig)

### 3.1.4 SARIMA Model

In [ ]:
import pmdarima as pm

sarima_model = pm.auto_arima(y=np.array(train[['Butler Trail - Crenshaw Bridge PC Urban Trail']]).reshape(len(train),),
                            exogenous=np.array(train[['TMAX', 'TMIN', 'PRCP']]).reshape(len(train), 3),
#                              start_p=1, 
#                              start_q=1,
#                              max_p=7, 
#                              max_q=7,
                             m=7,
#                              start_P=0,
#                              d=1, 
#                              D=1,
                             seasonal=True,
                             information_criterion='aic',
                             trace=True, 
                             suppress_warnings=True, 
                             stepwise=True
                            )

print(sarima_model.aic())

In [ ]:
print(sarima_model.summary())

In [100]:
sarima_score, sarima_scores, sarima_pred_list = arima_scoring(sarima_model, test)

/anaconda3/envs/ttf_hackathon/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.



IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
print('SARIMAX Model: [%.3f] %s' % (sarima_score, sarima_scores))

sarima_trace = go.Scatter(
    x=test.index[-7:], 
    y=sarima_pred_list[-1], 
    name='Forecasted Data'
)

test_trace = go.Scatter(
    x=test.index[-7:], 
    y=test['Butler Trail - Crenshaw Bridge PC Urban Trail'][-7:], 
    name='Actual Data'
)

data = [sarima_trace, test_trace]

layout = go.Layout(
    xaxis=dict(
        tickvals=test.index[-7:],
#         ticktext=days,
    ), 
    yaxis=dict(
        title='Count',
        rangemode='tozero'
    ), 
    title='SARIMAX Model Forecast'
)


fig = go.Figure(data=data, layout=layout)
iplot(fig)

mae_trace = go.Scatter(
    x=days, 
    y=sarima_scores,
    name='Mean Absolute Error'
)

data = [mae_trace]

layout = go.Layout(
    xaxis=dict(
        tickvals=days,
        ticktext=days,
        title='Days Forecast',
    ), 
    yaxis=dict(
        title='Mean Absolute Error',
        rangemode='tozero'
    ), 
    title='SARIMAX Model Performance'
)


fig = go.Figure(data=data, layout=layout)
iplot(fig)

### 3.1.5 Prophet Model

#### 3.1.5.1 Data Processing
I need to do some basic imports and get this data in the format Prophet expects.

In [ ]:
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics
from fbprophet.plot import plot_cross_validation_metric

import logging

logging.getLogger().setLevel(logging.ERROR)

In [ ]:
prophet_train_df = train[['Butler Trail - Crenshaw Bridge PC Urban Trail']]
prophet_train_df['ds'] = prophet_train_df.index
prophet_train_df.columns = ['y', 'ds']

print(prophet_train_df.tail(10))

prophet_test_df = test[['Butler Trail - Crenshaw Bridge PC Urban Trail']]
prophet_test_df['ds'] = prophet_test_df.index
prophet_test_df.columns = ['y', 'ds']

print(prophet_test_df.head(10))

Since ACL clearly has a big impact on trail traffic, we need to add it as a regressor. We can combine it with holidays. All of the events manually added below were cited by The Trail Foundation as especially busy days on the trail.


In [ ]:
festivals = pd.DataFrame(
    {
    'holiday': 'acl',
    'ds': pd.to_datetime(['2016-10-01', '2016-10-08', '2017-10-07', '2017-10-14', '2018-10-06', '2018-10-13',
                       '2019-10-05', '2019-10-12']),
    'lower_window': -2,
    'upper_window': 2
    }
)

lights = pd.DataFrame(
    {
    'holiday': 'lights',
    'ds': pd.to_datetime(['2016-12-10', '2017-12-10', '2018-12-10', '2019-12-10']),
    'lower_window': 0,
    'upper_window': 13
    }
)

garden = pd.DataFrame(
    {
    'holiday': 'garden',
    'ds': pd.to_datetime(['2016-04-02', '2017-03-25', '2018-03-24', '2019-03-23', '2020-03-22']),
    'lower_window': 0,
    'upper_window': 1
    }
)

eeyore = pd.DataFrame(
    {
    'holiday': 'eeyore',
    'ds': pd.to_datetime(['2016-04-30', '2017-04-29', '2018-04-28', '2019-04-27', '2020-04-26']),
    'lower_window': 0,
    'upper_window': 0
    }
)

kite = pd.DataFrame(
    {
    'holiday': 'kite',
    'ds': pd.to_datetime(['2016-03-06', '2017-03-05', '2018-03-04', '2019-03-31', '2020-03-28']),
    'lower_window': 0,
    'upper_window': 0
    }
)

festivals = festivals.append(lights)
festivals = festivals.append(garden)
festivals = festivals.append(eeyore)
festivals = festivals.append(kite)


festivals = festivals.reset_index()
festivals = festivals.drop(columns=['index'])

festivals

In [ ]:
m = Prophet(
#     changepoint_prior_scale=0.04, # default is 0.05, decreasing it makes trend more generalizable
    holidays=festivals,
)

m.add_country_holidays(country_name='US')
m.fit(prophet_df)
future = m.make_future_dataframe(periods=7)
forecast = m.predict(future)

In [ ]:
def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

def prophet_scoring(test_data, forecast_len=7):
    m = Prophet(
        holidays=festivals
    )
    m.add_country_holidays(country_name='US')
    m.fit(prophet_train_df)
    
    predictions_list = []
    future = m.make_future_dataframe(periods=7, include_history=False)
    forecast = m.predict(future)
    predictions_list.append(np.array(forecast['yhat']))
    for i in range(len(test_data) - forecast_len):
        m = Prophet(
            holidays=festivals
        )
        m.add_country_holidays(country_name='US')
        m.fit(prophet_train_df.append(prophet_test_df.iloc[[i]]))
        future = m.make_future_dataframe(periods=7, include_history=False)
        forecast = m.predict(future)
        predictions_list.append(np.array(forecast['yhat']))
    # calculate mae
    test_weeks = rolling_window(test['Butler Trail - Crenshaw Bridge PC Urban Trail'], 7)
    predictions_array = np.array(predictions_list)
    scores = []
    for i in range(test_weeks.shape[1]):
        mae = mean_absolute_error(test_weeks[:, i],
                                  predictions_array[:, i])
        scores.append(mae)
    # calculate overall MAE
    s = 0
    for row in range(test_weeks.shape[0]):
        for col in range(test_weeks.shape[1]):
            s += np.abs((test_weeks[row, col] - predictions_array[row, col]))
    score = (s / (test_weeks.shape[0] * test_weeks.shape[1]))
    return score, scores, predictions_list

In [ ]:
m = Prophet(
    holidays=festivals,
)
m.add_country_holidays(country_name='US')
m.fit(prophet_train_df)

predictions_list = []
future = m.make_future_dataframe(periods=7, include_history=True)
forecast = m.predict(future)
predictions_list.append(np.array(forecast['yhat']))

In [ ]:
m.plot_components(forecast)

In [ ]:
prophet_score, prophet_scores, prophet_pred_list = prophet_scoring(prophet_test_df)

In [ ]:
print('Prophet Model: [%.3f] %s' % (prophet_score, prophet_scores))


prophet_trace = go.Scatter(
    x=test.index[-7:], 
    y=prophet_pred_list[-1],
    name='Forecasted Data'
)

actual_trace = go.Scatter(
    x=test.index[-7:], 
    y=prophet_test_df['y'][-7:],
    name='Actual Data'
)

data = [prophet_trace, actual_trace]

layout = go.Layout(
    xaxis=dict(
        tickvals=test.index[-7:],
#         ticktext=days,
    ), 
    yaxis=dict(
        title='Count',
        rangemode='tozero'
    ), 
    title='Prophet Model Forecast'
)


fig = go.Figure(data=data, layout=layout)
iplot(fig)


mae_trace = go.Scatter(
    x=days, 
    y=prophet_scores,
    name='Mean Absolute Error'
)

data = [mae_trace]

layout = go.Layout(
    xaxis=dict(
        tickvals=days,
        ticktext=days,
        title='Days Forecast',
    ), 
    yaxis=dict(
        title='Mean Absolute Error',
        rangemode='tozero'
    ), 
    title='Prophet Model Performance'
)


fig = go.Figure(data=data, layout=layout)
iplot(fig)

### 3.1.6 Prophet Model With Additional Regressors

In [ ]:
prophet_train_df = train[['Butler Trail - Crenshaw Bridge PC Urban Trail', 'TMAX', 'TMIN', 'PRCP']]
prophet_train_df['ds'] = prophet_train_df.index
prophet_train_df.columns = ['y', 'TMAX', 'TMIN', 'PRCP', 'ds']

print(prophet_train_df.tail(10))

prophet_test_df = test[['Butler Trail - Crenshaw Bridge PC Urban Trail', 'TMAX', 'TMIN', 'PRCP']]
prophet_test_df['ds'] = prophet_test_df.index
prophet_test_df.columns = ['y', 'TMAX', 'TMIN', 'PRCP', 'ds']

print(prophet_test_df.head(10))

In [ ]:
def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

def prophet_weather_scoring(train_data, test_data, forecast_len=7):
#     m = Prophet(
#         holidays=festivals
#     )
#     m.add_regressor('TMAX')
#     m.add_regressor('TMIN')
#     m.add_regressor('PRCP')
#     m.add_country_holidays(country_name='US')
#     m.fit(train_data)

    predictions_list = []
#     future = m.make_future_dataframe(periods=7, include_history=False)
#     future['TMAX'] = np.array(prophet_test_df['TMAX'][:7])
#     future['TMIN'] = np.array(prophet_test_df['TMIN'][:7])
#     future['PRCP'] = np.array(prophet_test_df['PRCP'][:7])
#     forecast = m.predict(future)
#     predictions_list.append(np.array(forecast['yhat']))
    for i in range(len(test_data) - forecast_len + 1):
        m = Prophet(
            holidays=festivals
        )
        m.add_regressor('TMAX')
        m.add_regressor('TMIN')
        m.add_regressor('PRCP')
        m.add_country_holidays(country_name='US')
        if i == 0:
            m.fit(prophet_train_df)
        else:
            m.fit(prophet_train_df.append(prophet_test_df.iloc[[i]])) 
        future = m.make_future_dataframe(periods=7, include_history=False)
        future['TMAX'] = np.array(prophet_test_df['TMAX'][i:i + forecast_len])
        future['TMIN'] = np.array(prophet_test_df['TMIN'][i:i + forecast_len])
        future['PRCP'] = np.array(prophet_test_df['PRCP'][i:i + forecast_len])
        forecast = m.predict(future)
        predictions_list.append(np.array(forecast['yhat']))
    # calculate mae
    test_weeks = rolling_window(test_data['y'], 7)
    predictions_array = np.array(predictions_list)
    scores = []
    for i in range(test_weeks.shape[1]):
        mae = mean_absolute_error(test_weeks[:, i],
                                  predictions_array[:, i])
        scores.append(mae)
    # calculate overall MAE
    s = 0
    for row in range(test_weeks.shape[0]):
        for col in range(test_weeks.shape[1]):
            s += np.abs((test_weeks[row, col] - predictions_array[row, col]))
    score = (s / (test_weeks.shape[0] * test_weeks.shape[1]))
    return score, scores, predictions_list

In [ ]:
m = Prophet(
    holidays=festivals
)
m.add_regressor('TMAX')
m.add_regressor('TMIN')
m.add_regressor('PRCP')
m.add_country_holidays(country_name='US')
m.fit(prophet_train_df)

future = m.make_future_dataframe(periods=7, include_history=True)
future['TMAX'] = np.array(prophet_train_df['TMAX'].append(prophet_test_df['TMAX'][:7]))
future['TMIN'] = np.array(prophet_train_df['TMIN'].append(prophet_test_df['TMIN'][:7]))
future['PRCP'] = np.array(prophet_train_df['PRCP'].append(prophet_test_df['PRCP'][:7]))
forecast = m.predict(future)
predictions_list.append(np.array(forecast['yhat']))

In [ ]:
m.plot_components(forecast)

In [ ]:
prophet_exog_score, prophet_exog_scores, prophet_exog_pred_list = prophet_weather_scoring(prophet_train_df, prophet_test_df)

In [ ]:
print('Prophet Model: [%.3f] %s' % (prophet_exog_score, prophet_exog_scores))


prophet_trace = go.Scatter(
    x=test.index[-7:], 
    y=prophet_exog_pred_list[-1],
    name='Forecasted Data'
)

actual_trace = go.Scatter(
    x=test.index[-7:], 
    y=prophet_test_df['y'][-7:],
    name='Actual Data'
)

data = [prophet_trace, actual_trace]

layout = go.Layout(
    xaxis=dict(
        tickvals=test.index[-7:],
#         ticktext=days,
    ), 
    yaxis=dict(
        title='Count',
        rangemode='tozero'
    ), 
    title='Prophet Model (+ Weather) Forecast'
)


fig = go.Figure(data=data, layout=layout)
iplot(fig)


mae_trace = go.Scatter(
    x=days, 
    y=prophet_exog_scores,
    name='Mean Absolute Error'
)

data = [mae_trace]

layout = go.Layout(
    xaxis=dict(
        tickvals=days,
        ticktext=days,
        title='Days Forecast',
    ), 
    yaxis=dict(
        title='Mean Absolute Error',
        rangemode='tozero'
    ), 
    title='Prophet Model Performance'
)


fig = go.Figure(data=data, layout=layout)
iplot(fig)

# 4. Evaluation

I'd like to compare the MAEs of each model across the seven day forecasts as well as overall.

In [ ]:
mean_maes_trace = go.Scatter(
    x=days, 
    y=mean_scores,
    name='Mean Model'
)

sine_maes_trace = go.Scatter(
    x=days, 
    y=sine_scores,
    name='Sine Model'
)

arima_maes_trace = go.Scatter(
    x=days, 
    y=arima_scores,
    name='ARIMA Model'
)

sarima_maes_trace = go.Scatter(
    x=days, 
    y=sarima_scores,
    name='SARIMAX Model'
)

prophet_maes_trace = go.Scatter(
    x=days, 
    y=prophet_scores,
    name='Prophet Model'
)

prophet_exog_maes_trace = go.Scatter(
    x=days, 
    y=prophet_exog_scores,
    name='Prophet + Weather Model'
)

data = [mean_maes_trace, sine_maes_trace, arima_maes_trace, sarima_maes_trace, prophet_maes_trace, prophet_exog_maes_trace]

layout = go.Layout(
    xaxis=dict(
        tickvals=days,
        ticktext=days,
        title='Forecast Horizon',
    ), 
    yaxis=dict(
        title='Mean Absolute Error',
        rangemode='tozero'
    ), 
    title='MAE Scores By Forecasting Horizon'
)


fig = go.Figure(data=data, layout=layout)
iplot(fig)

model_names = ['Mean', 'Sine Wave', 'ARIMA', 'SARIMAX', 'Prophet', 'Prophet +Weather']
score_list = [mean_score, sine_score, arima_score, sarima_score, prophet_score, prophet_exog_score]

mean_mae_trace = go.Bar(
    x=model_names, 
    y=score_list,
    name='Overall MAE',
    marker={'color':['blue', 'orange', 'green', 'red', 'purple', 'brown']},
    text=[int(score) for score in score_list],
    textposition='auto'
)

data = [mean_mae_trace]

layout = go.Layout(
    xaxis=dict(
        tickvals=model_names,
        ticktext=model_names,
        title='Model Type',
    ), 
    yaxis=dict(
        title='Mean Absolute Error',
        rangemode='tozero'
    ), 
    title='Overall MAE'
)


fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
fig2 = m.plot_components(forecast)

In [ ]:
df_cv = cross_validation(m, horizon='21 days')
# df_p = performance_metrics(df_cv)

In [ ]:
fig3 = plot_cross_validation_metric(df_cv, metric='mae', figsize=(14, 7))

I'll now scatter plot the data to see if there are outliers I can't explain:

In [ ]:
x = prophet_df['ds']
y = prophet_df['y']

trace1 = go.Scatter(x=x, y=y, name='Butler-Crenshaw', mode='markers')

data = [trace1]
layout = go.Layout(title='Daily Total Traffic (Butler-Crenshaw)')

fig = go.Figure(data=data, layout=layout)
iplot(fig)

It looks like we have clear outlier data at a few points.
* March 6, 2016
* Early to mid-October every year in the data

After researching the dates, I can't find a reason for so much traffic on March 6 and suspect it is an error. Even if it isn't, it doesn't come up in subsequent years so probably shouldn't be modeled.
The other outlier dates are around ACL (two weekends in October annually). It makes sense that we would see a large increase in traffic around that time given the [location of this counter](https://www.google.com/maps/place/30%C2%B016'28.3%22N+97%C2%B046'16.4%22W/@30.27452,-97.7727358,17z/data=!3m1!4b1!4m5!3m4!1s0x0:0x0!8m2!3d30.27452!4d-97.77122?hl=en). We will want to make sure to model those dates as a holiday.

In [ ]:
prophet_df.loc[prophet_df['ds'] == '2016-03-06', 'y'] = None

It would be helpful to incorporate weather. We can say with confidence that there will be less people on the trail on days where it's very cold, hot, or rainy. I downloaded a dataset from [NOAA](https://www.ncdc.noaa.gov/cdo-web/search) that has daily weather summaries for Austin for the last 10 years (May 11 2009 - May 11 2019). My plan is to use the average high for each day of the year in that dataset as the temperature for that calendar day in the future. I will do the same for rain. That way, I can add temperatures and presence of rain as additional regressors to this model.

In [ ]:
trail_df

In [ ]:
weather_df = pd.read_csv('weather_7-9-19.csv')
weather_df = weather_df[['DATE','TMAX', 'TMIN', 'PRCP']]
weather_df['DATE'] = pd.to_datetime(weather_df['DATE'])

In [ ]:
weather_df.tail()

Now I want mean high/low temps and rainfall for each day of the calendar year. I can regroup this to give me that.

In [ ]:
month_day_weather_df = weather_df.groupby([(weather_df['DATE'].dt.month),(weather_df['DATE'].dt.day)]).mean()

In [ ]:
month_day_weather_df.head()

* Extend this out to the future with averages
* Merge it with my original dataframe starting at the start date for trail counts
* Create columns showing max temperature, min temperature (highs and lows) and precipitation
* Create a Prophet dataframe handling these as additional regressors

In [ ]:
past_weather_df = trail_df.merge(weather_df, how='inner', left_on=['Date'], right_on=['DATE'])
past_weather_df = past_weather_df.drop(columns=['DATE'])
past_weather_df = past_weather_df[['Date', 'TMAX', 'TMIN', 'PRCP']]
past_weather_df = past_weather_df.set_index('Date')

past_weather_df.tail()

Now we have a DataFrame, past_weather_df, that gives us mean daily highs and lows based on ten years of weather data. We also have a column of binary values indicating whether there was significant precipitation each day.

We need one for predictions about future weather. This new df will predict any day where there has been an average of >= .1 inch of rain (considered moderate rain) to be rainy in the future.

In [ ]:
future_weather_df = weather_df.copy()
future_weather_df = future_weather_df[:365]
future_weather_df['DATE'] = pd.date_range(start='7/2/2019', freq='D', periods=365)
future_weather_df['Month'] = future_weather_df['DATE'].dt.month
future_weather_df['Day'] = future_weather_df['DATE'].dt.day
future_weather_df = future_weather_df.drop(columns=['TMAX', 'TMIN', 'PRCP'])
future_weather_df = future_weather_df.merge(month_day_weather_df, how='left', left_on=['Month', 'Day'], right_index=True)
future_weather_df = future_weather_df.drop(columns=['Month', 'Day'])
future_weather_df = future_weather_df.set_index('DATE')

future_weather_df.head()

Now we have another DataFrame, future_weather_df, that gives us one year of predictions for daily highs, lows, and precipitation based on means over the ten years of weather data we have.

I need to split them into 3 different dataframes for past and future so Prodigy can use them each as a regressor.

In [ ]:
past_high_df = past_weather_df[['TMAX']]
past_low_df = past_weather_df[['TMIN']]
past_rain_df = past_weather_df[['PRCP']]

future_high_df = future_weather_df[['TMAX']]
future_low_df = future_weather_df[['TMIN']]
future_rain_df = future_weather_df[['PRCP']]

In [ ]:
prophet_df = prophet_df.merge(past_high_df, how='right', left_on=['ds'], right_on=['Date'])
prophet_df = prophet_df.merge(past_low_df, how='right', left_on=['ds'], right_on=['Date'])
prophet_df = prophet_df.merge(past_rain_df, how='right', left_on=['ds'], right_on=['Date'])

prophet_df

In [ ]:
# I'll just use the festivals I defined above.

m = Prophet(
#     interval_width=0.8,
    changepoint_prior_scale=0.03, # default is 0.05, decreasing it makes trend more generalizable
    holidays=festivals,
#     holidays_prior_scale=20, # default is 10, strength of the holiday components
#     yearly_seasonality=15,
    weekly_seasonality=7, # trail usage patterns change by day of the week
# #     seasonality_prior_scale=20
)
m.add_regressor('TMAX', prior_scale=0.5, mode='multiplicative')
m.add_regressor('TMIN', prior_scale=0.5, mode='multiplicative')
m.add_regressor('PRCP', prior_scale=0.5, mode='multiplicative')
m.add_country_holidays(country_name='US')
m.fit(prophet_df)


In [ ]:
all_high_df = past_high_df.append(future_high_df)
all_low_df = past_low_df.append(future_low_df)
all_rain_df = past_rain_df.append(future_rain_df)

In [ ]:
future = m.make_future_dataframe(periods=21)
future = future.merge(all_high_df, how='left', left_on=['ds'], right_index=True)
future = future.merge(all_low_df, how='left', left_on=['ds'], right_index=True)
future = future.merge(all_rain_df, how='left', left_on=['ds'], right_index=True)
forecast = m.predict(future)
fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)

In [ ]:
df_cv = cross_validation(m, horizon='21 days')
df_p = performance_metrics(df_cv)

fig3 = plot_cross_validation_metric(df_cv, metric='mae')

I'm going to create a baseline model here that will just predict the average of the last 21 days. I'll then compare this to the model I created and see if I'm outperforming it. Hopefully I am.

In [ ]:
naive_y_df = prophet_df.copy()
naive_y_df.head()

In [ ]:
naive_y_df['yhat'] = naive_y_df['y'].rolling(21).mean()

In [ ]:
naive_y_df.tail()

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
naive_y_df.loc[144]

In [ ]:
naive_y_df = naive_y_df[['ds', 'y', 'yhat']][144:]
naive_y_df = naive_y_df.dropna(thresh=3)

y_true = naive_y_df['y']
y_pred = naive_y_df['yhat']
mean_absolute_error(y_true, y_pred)

In [ ]:
naive_y_df

In [ ]:
df_cv

In [ ]:
mae_df_cv = df_cv[['ds', 'y', 'yhat']]
mae_df_cv = mae_df_cv.dropna(thresh=3)
# mae_df_cv
y_true = mae_df_cv['y']
y_pred = mae_df_cv['yhat']
mean_absolute_error(y_true, y_pred)

In [ ]:
# forecast horizon
H = 7

# frequency of simulated forecasts
h = int(H/2)

# tuning and validation: simulated historical forecast

# for storing forecast results and cutoffdates
results = pd.DataFrame()
cutoff = []

# run forecast simulations
i = 0
while (len(prophet_df) - i > 3 * H): 

    # define training data
    train = prophet_df[i:(i + (3 * H))] # use 3 periods of data for training

    # fit time series model
    m_test = Prophet(interval_width=0.8,
                changepoint_prior_scale=0.05, # default is 0.05, decreasing it makes trend more generalizable
                holidays=festivals,
                holidays_prior_scale=15, # default is 10, strength of the holiday components
                yearly_seasonality=True,
                weekly_seasonality=True, # trail usage patterns change by day of the week
                seasonality_prior_scale=20, # default is 10, larger values allow larger seasonal fluctuations
#                 mcmc_samples=500 # to generate confidence intervals for seasonality and holiday components
               )
    m_test.add_regressor('TMAX')
    m_test.add_regressor('TMIN')
    m_test.add_regressor('PRCP')
    m_test.add_country_holidays(country_name='US')
    m_test.fit(train);
    
    # future dates for which to make forecasts
    future = m_test.make_future_dataframe(periods=H)
    future = future.merge(all_high_df, how='left', left_on=['ds'], right_index=True)
    future = future.merge(all_low_df, how='left', left_on=['ds'], right_index=True)
    future = future.merge(all_rain_df, how='left', left_on=['ds'], right_index=True)
    
    # make forecast
    forecast = m_test.predict(future)
    resultsH = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(H)
    
    # get actual values to compare with predicted values
    resultsH = prophet_df.merge(resultsH, how='right')
        
    # sort by increasing date
    resultsH = resultsH.sort_values(by='ds')
    
    # record cutoff dates
    cutoffDate = resultsH['ds'].iloc[0]
    cutoffDate = cutoffDate.strftime('%Y %b')
    cutoff = cutoff + [cutoffDate]
    
    # compile results
    results = pd.concat((results, resultsH))
    
    print('Counting the days...', i)
    i = i + h

### This needs to be rewritten to take in multivariate data and use the features as regressors. Currently stripping everything but count and using ordinal position as independent var - treating as univariate problem. Needs to accept weather. Recursive forecasting probably won't be the best approach since it would need to predict multiple features in order to append predictions to subsequent steps of history and make new predictions. Direct multi-step forecast is probably the best approach since a suite of models can each predict one dependent variable (count) using the three weather features as independent variables.

In [ ]:
# direct multi-step forecast by lead time
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import Lars
from sklearn.linear_model import LassoLars
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import SGDRegressor

# split a univariate dataset into train/test sets
def split_dataset(data):
    # split into standard weeks
    train, test = data[2:-76], data[-76:-6]
    # restructure into windows of weekly data
    train = array(split(train, len(train)/7))
    test = array(split(test, len(test)/7))
    return train, test

# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
	scores = list()
	# calculate an RMSE score for each day
	for i in range(actual.shape[1]):
		# calculate mae
		mae = mean_absolute_error(actual[:, i], predicted[:, i])
		# calculate rmse
# 		rmse = sqrt(mse)
		# store
		scores.append(mae)
	# calculate overall MAE
	s = 0
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
# 			s += (actual[row, col] - predicted[row, col])**2
			s += np.abs((actual[row, col] - predicted[row, col]))
	score = (s / (actual.shape[0] * actual.shape[1]))
	return score, scores

# summarize scores
def summarize_scores(name, score, scores):
	s_scores = ', '.join(['%.1f' % s for s in scores])
	print('%s: [%.3f] %s' % (name, score, s_scores))

# prepare a list of ml models
def get_models(models=dict()):
	# linear models
	models['lr'] = LinearRegression()
	models['lasso'] = Lasso()
	models['ridge'] = Ridge()
	models['en'] = ElasticNet()
	models['huber'] = HuberRegressor()
	models['lars'] = Lars()
	models['llars'] = LassoLars()
	models['pa'] = PassiveAggressiveRegressor(max_iter=1000, tol=1e-3)
	models['ranscac'] = RANSACRegressor()
	models['sgd'] = SGDRegressor(max_iter=1000, tol=1e-3)
	print('Defined %d models' % len(models))
	return models

# create a feature preparation pipeline for a model
def make_pipeline(model):
	steps = list()
	# standardization
	steps.append(('standardize', StandardScaler()))
	# normalization
	steps.append(('normalize', MinMaxScaler()))
	# the model
	steps.append(('model', model))
	# create pipeline
	pipeline = Pipeline(steps=steps)
	return pipeline

# # convert windows of weekly multivariate data into a series of total power
def to_series(data):
	# extract just the total power from each week
	series = [week for week in data]
	# flatten into a single series
	series = array(series).flatten()
	return series

# convert history into inputs and outputs
def to_supervised(history, n_input, output_ix):
	# convert history to a univariate series
	data = to_series(history)
	X, y = list(), list()
	ix_start = 0
	# step over the entire history one time step at a time
	for i in range(len(data)):
		# define the end of the input sequence
		ix_end = ix_start + n_input
		ix_output = ix_end + output_ix
		# ensure we have enough data for this instance
		if ix_output < len(data):
			X.append(data[ix_start:ix_end])
			y.append(data[ix_output])
		# move along one time step
		ix_start += 1
	return array(X), array(y)

# fit a model and make a forecast
def sklearn_predict(model, history, n_input):
	yhat_sequence = list()
	# fit a model for each forecast day
	for i in range(7):
		# prepare data
		train_x, train_y = to_supervised(history, n_input, i)
		# make pipeline
		pipeline = make_pipeline(model)
		# fit the model
		pipeline.fit(train_x, train_y)
		# forecast
		x_input = array(train_x[-1, :]).reshape(1,n_input)
		yhat = pipeline.predict(x_input)[0]
		# store
		yhat_sequence.append(yhat)
	return yhat_sequence

# evaluate a single model
def evaluate_model(model, train, test, n_input):
	# history is a list of weekly data
	history = [x for x in train]
	# walk-forward validation over each week
	predictions = list()
	for i in range(len(test)):
		# predict the week
		yhat_sequence = sklearn_predict(model, history, n_input)
		# store the predictions
		predictions.append(yhat_sequence)
		# get real observation and add to history for predicting the next week
		history.append(test[i, :])
	predictions = array(predictions)
	# evaluate predictions days for each week
	score, scores = evaluate_forecasts(test[:, :, 0], predictions)
	return score, scores

pyplot.figure(figsize=(14, 7))

# load the new file
dataset = multivariate_df
# split into train and test
train, test = split_dataset(dataset.values)
# prepare the models to evaluate
models = get_models()
n_input = 7
# evaluate each model
days = ['+1', '+2', '+3', '+4', '+5', '+6', '+7']
for name, model in models.items():
	# evaluate and get scores
	score, scores = evaluate_model(model, train, test, n_input)
	# summarize scores
	summarize_scores(name, score, scores)
	# plot scores
	pyplot.plot(days, scores, marker='o', label=name)
# show plot
pyplot.xlabel('Days Out')
pyplot.ylabel('Mean Absolute Error')
pyplot.yticks(ticks=[0, 500, 1000, 1500, 2000, 2500, 3000])
pyplot.legend()
pyplot.show()